In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.regularizers import l2
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import regularizers
from tensorflow.python.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import MultiHeadAttention
from keras.layers import Input
from keras.layers import Concatenate,GlobalAveragePooling1D
from keras.layers import Masking, LSTM, TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import smote_variants as sv

In [2]:
data1 = pd.read_csv("dataset/xerces12.csv")
(row, cell) = data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [3]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row,cell-1,1)
X_train, y_train = X_res, y_res

In [4]:
data1 = pd.read_csv("dataset/xerces13.csv")
(row1, cell1) = data1.values.shape
data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [5]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row1,cell-1,1)
X_test, y_test = X_res, y_res

In [6]:
# 转换标签为独热编码
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((731, 20, 1), (445, 20, 1), (731, 2), (445, 2))

In [7]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
threshold = 0.5  

def mcc_metric(y_true, y_pred):
    predicted = tf.cast(tf.greater(y_pred, threshold), tf.float32)
    true_pos = tf.math.count_nonzero(predicted * y_true)
    true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
    false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
    false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
    x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
        * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
    return tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)


In [8]:
num_classes=2
from keras import regularizers
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=4 , input_shape=(cell-1,1),activation= 'relu', kernel_regularizer=regularizers.l2(0.001) ))
    model.add(MaxPooling1D(pool_size=2))
    #model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(21, activation= 'relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dense(num_classes, activation= 'sigmoid', kernel_regularizer=regularizers.l2(0.001) ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy' ,f1_m,recall_m,mcc_metric])
    return model

In [9]:

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [10]:
# build the model
model = baseline_model()
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=50,verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[3]*100))



Epoch 1/50


G:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.2074 - f1_m: 0.3131 - loss: 29.2113 - mcc_metric: -0.5039 - recall_m: 0.3408 - val_accuracy: 0.1124 - val_f1_m: 0.2728 - val_loss: 10.7995 - val_mcc_metric: -0.6388 - val_recall_m: 0.3054
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2354 - f1_m: 0.3972 - loss: 17.4296 - mcc_metric: -0.4014 - recall_m: 0.4583 - val_accuracy: 0.1685 - val_f1_m: 0.4022 - val_loss: 6.2206 - val_mcc_metric: -0.3863 - val_recall_m: 0.4608
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3303 - f1_m: 0.5109 - loss: 6.3061 - mcc_metric: -0.1701 - recall_m: 0.6064 - val_accuracy: 0.7079 - val_f1_m: 0.6795 - val_loss: 6.8328 - val_mcc_metric: 0.2572 - val_recall_m: 0.8074
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7622 - f1_m: 0.6821 - loss: 5.8627 - mcc_metric: 0.2816 - recall_m: 0.7870 - val_accuracy: 0.8202 - val_f1_m: 0.7498 - val_loss: 6.9032 - val_mcc_metric: 0.4858 - val_recall_m: 0.7674
Epoch 5/50
8/8 ━━━

In [13]:
print(" Loss               Accuracy            F1 score             Recall              Mcc         ")
print(scores)


 Loss               Accuracy            F1 score             Recall              Mcc         
[0.5486233830451965, 0.8539325594902039, 0.8361876606941223, 0.8604166507720947, 0.6614089012145996]
